In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time

In [2]:
def get_webpage_section(soup, tag, attr_type, val): # 濾出需要的區塊
    if attr_type == "id":
        result = soup.find(tag, id=val)
    elif attr_type == "class":
        result = soup.find_all(tag, class_=val) # class 是保留字, bs4 用 class_
    return result

In [3]:
def get_movie_pages(menuUrl): # moviePages: 電影資訊的 List [ChineseName, Url]
    moviePages = list()
    while True:
        r = requests.get(menuUrl)
        soup = BeautifulSoup(r.text, "html.parser")
        movieMenu = get_webpage_section(soup, "div", "class", "release_movie_name")
        for movie in movieMenu:
            # print(movie.select("a"))
            movieChiName = re.sub(r'\s', '', movie.select("a")[0].text)
            # movieEngName = re.sub(r'\s', '', movie.select("a")[1].text) # 進電影資訊頁面再爬
            movieUrl = movie.select("a")[0]['href']
            moviePages.append([movieChiName, movieUrl])
        ### 掃每一頁
        nextPage = get_webpage_section(soup, "li", "class", "nexttxt")
        if nextPage != [] and nextPage[0].find("a") != None: # 若下一頁存在, 更新 menuUrl, loop 繼續抓
            menuUrl = nextPage[0].find("a")['href']
            # print(menuUrl)
        else:
            break
    return moviePages

In [26]:
def get_movie_info(movieInfo): # movieInfo: [ChineseName, Url] # 濾出 分類, 上映時間, 劇情介紹
    r = requests.get(movieInfo[1]) # movieInfo[1]: 電影資訊頁面的網址
    soup = BeautifulSoup(r.text, "html.parser")
    # 英文名稱
    englishNameSec = get_webpage_section(soup, "div", "class", "movie_intro_info_r")
    if englishNameSec != None and englishNameSec != []:
        englishName = englishNameSec[0].find("h3").text # englishName = englishNameSec[0].find("h3").text
    else:
        englishName = 'None'
    movieInfo.append(re.sub(r'\s', '', englishName))
    # 分類
    movieClassSec = get_webpage_section(soup, "div", "class", "level_name_box")
    if movieClassSec != None and movieClassSec != []:
        movieInfo.append([re.sub(r'\s', '', movieCls.text) for movieCls in movieClassSec[0].select("a")])
    else:
        movieInfo.append(['None'])
    # 上映時間
    moviePage = get_webpage_section(soup, "div", "class", "movie_intro_info_r")
    if moviePage != None and movieClassSec != []
    movieInfo.append(moviePage[0].find("span").text)
    # 劇情介紹
    movieStory = get_webpage_section(soup, "span", "id", "story")
    movieInfo.append(re.sub(r'\s', '', movieStory.text))
    
    return movieInfo

In [5]:
def pack_movie_info(movieList): # to json
    movieInfoList = list()
    for movie in movieList: # movie: [Name, Url]
        movieInfo = get_movie_info(movie) 
        movieInfoList.append(movieInfo[:1] + movieInfo[2:]) # 不需要 Url (movieInfo[1])
    movieJson = dict()
    for idx in range(len(movieInfoList)): # movieInfoList: [Name, Name, category, releaseTime, storyIntro]
        mov_dict = dict()
        for jdx in range(len(movieInfoList[idx])):
            if jdx == 0: mov_dict['Chinese Name'] = movieInfoList[idx][jdx]
            elif jdx == 1: mov_dict['English Name'] = movieInfoList[idx][jdx]
            elif jdx == 2: mov_dict['Movie Categories'] = movieInfoList[idx][jdx]
            elif jdx == 3: mov_dict['Release Date'] = movieInfoList[idx][jdx][5:]
            elif jdx == 4: mov_dict['Storyline Intro'] = movieInfoList[idx][jdx]
        movieJson[ str(idx) ] = mov_dict
    # print(movieJson)
    return movieInfoList, movieJson

In [6]:
def get_actors(movieList, ActorSet): # movieList: [[ChineseName, Url], ...] # 濾出 演員 *********
    ActorsUrlList = list()
    for movie in movieList: # movie: [Name, Url]
        if len(ActorSet) >= 2500:
            break
        r = requests.get(movie[1])
        soup = BeautifulSoup(r.text, "html.parser")
        # 演員 Url
        movieActors = get_webpage_section(soup, "ul", "class", "starlist")
        if movieActors != []:
            # ActorsUrlList += [actor['href'] for actor in movieActors[0].find_all("a")]
            for actor in movieActors[0].find_all("a"):
                if actor not in ActorSet:
                    ActorSet.add(actor)
                    ActorsUrlList.append(actor['href'])
    return ActorsUrlList

In [7]:
def get_movie_url_from_actors(actorsUrl, movieSet): # , existMovieSet):
    moviePages = list()
    r = requests.get(actorsUrl)
    soup = BeautifulSoup(r.text, "html.parser")
    movieList = get_webpage_section(soup, "ul", "class", "trailer_list")
    for movie in movieList: 
        # moviePages += [[re.sub(r'\s', '', m.text), m['href']] for m in movie.select("a")]
        for m in movie.select("a"):
            newMovie = [re.sub(r'\s', '', m.text), m['href']]
            if tuple(newMovie) not in movieSet:
                moviePages.append(newMovie)
                movieSet.add(tuple(newMovie))
    return moviePages

In [27]:
def main():
    start = time.time() # Debug 用
    r = requests.get("https://movies.yahoo.com.tw/")
    soup = BeautifulSoup(r.text, "html.parser")
    sel = get_webpage_section(soup, "ul", "id", "mainmenu").select("a")
    mainmenu = dict()
    # 爬下這三個 menu 的網址
    for s in sel:
        for t in ['本週新片', '上映中', '即將上映']: 
            if t in s.text: # 是以上這些種類的, 連結存放到 main menu
                mainmenu[t] = s['href'] 
    movieList = list()
    # 掃 menu 的電影, 爬下電影資訊的網址
    for movie, url in mainmenu.items():
        movieList += get_movie_pages(url)
    movieSet = set(tuple(movie) for movie in movieList)
    # movieInfoList, movieJson = pack_movie_info(movieList) # movieInfoList: Pure List, movieJson: Json for outfile
    # print(movieList)
    ActorSet = set()
    ### 演員其他作品第一層
    movieActorsUrlList = get_actors(movieList, ActorSet)
    movieUrlListFromActors = list()
    print("Start getting movies from Actors...")
    for url in movieActorsUrlList:
        movieUrlListFromActors += get_movie_url_from_actors(url, movieSet) # 從演員取得電影網址 (已有的不加)
        print(str(len(movieUrlListFromActors)) + "...")
    # movieInfoList_1, movieJson_1 = pack_movie_info(movieUrlListFromActors) 
    ### 演員其他作品第二層
    movieActorsUrlList_2 = get_actors(movieUrlListFromActors, ActorSet)
    movieUrlListFromActors_2 = list()
    print("Start getting movies from Actors...")
    for url in movieActorsUrlList_2:
        movieUrlListFromActors_2 += get_movie_url_from_actors(url, movieSet) # 從演員取得電影網址 (已有的不加)
        print(str(len(movieUrlListFromActors_2)) + "...")
    # print(movieList + movieUrlListFromActors + movieUrlListFromActors_2[-1])
    movieInfoList, movieJson = pack_movie_info(movieList + movieUrlListFromActors + movieUrlListFromActors_2)
    
    file = open('movie_info.txt', 'w')
    json.dump(movieJson, file, indent=4, ensure_ascii=False)
    file.close()
    
    end = time.time() # Debug 用
    print("Time: %f 秒" % (end - start))
main()

Start getting movies from Actors...
21...
45...
54...
74...
81...
82...
105...
108...
120...
137...
139...
150...
151...
157...
165...
165...
169...
169...
175...
179...
200...
219...
221...
222...
235...
237...
249...
253...
266...
276...
289...
298...
320...
331...
332...
345...
348...
351...
353...
353...
353...
353...
353...
359...
361...
364...
369...
385...
414...
418...
426...
430...
430...
430...
431...
451...
468...
473...
480...
489...
492...
497...
508...
520...
538...
539...
540...
541...
557...
566...
569...
585...
586...
608...
609...
626...
644...
653...
660...
667...
689...
695...
709...
709...
709...
710...
710...
727...
732...
749...
760...
765...
771...
776...
779...
780...
781...
793...
798...
807...
810...
813...
819...
824...
829...
833...
854...
879...
883...
883...
892...
894...
895...
909...
928...
929...
936...
954...
954...
960...
973...
975...
976...
976...
977...
979...
982...
983...
988...
997...
997...
997...
1007...
1031...
1052...
1054...
1054...
1057..

2650...
2650...
2650...
2650...
2652...
2653...
2659...
2661...
2663...
2664...
2666...
2666...
2667...
2667...
2667...
2669...
2669...
2669...
2673...
2675...
2675...
2675...
2676...
2677...
2677...
2677...
2678...
2683...
2685...
2687...
2687...
2687...
2691...
2691...
2691...
2693...
2695...
2697...
2698...
2700...
2700...
2703...
2703...
2704...
2705...
2707...
2707...
2707...
2708...
2710...
2710...
2710...
2710...
2715...
2715...
2715...
2717...
2717...
2718...
2722...
2724...
2729...
2731...
2731...
2731...
2732...
2734...
2739...
2739...
2739...
2742...
2744...
2744...
2745...
2745...
2746...
2748...
2749...
2754...
2754...
2760...
2760...
2760...
2760...
2760...
2764...
2765...
2765...
2765...
2765...
2766...
2766...
2766...
2766...
2766...
2767...
2768...
2768...
2770...
2770...
2771...
2771...
2774...
2774...
2774...
2774...
2774...
2775...
2777...
2778...
2779...
2780...
2782...
2783...
2783...
2783...
2783...
2784...
2784...
2784...
2784...
2784...
2785...
2788...
2789...


IndexError: list index out of range

In [18]:
file = open("movie_info.txt", "r")
print(file.read())
file.close()

In [49]:
li = []
if li == []:
    print(True)
else:
    print(False)

True


In [126]:
s = set()
s.add(1)
s.add(2)
if 1 in s:
    print("yes")
print(s)

yes
{1, 2}


In [180]:
nums = [[-4, [0, 1], 2], [3, 4, 5, 6], [-4, 0, 1, 2]]
s = set(tuple(l) for l in nums)
s
if tuple([-4, [0, 1], 2]) in s:
    print("yes")

TypeError: unhashable type: 'list'